# Train Networks

- Three-way SoftMax or Multi-BCE classifier of normal, non-vascular MCI, and non-vascular dementia

-----

## Load Packages

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
# Load some packages
import os
import json
from copy import deepcopy

import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pprint
import wandb

# custom package
from models import *
from utils.eeg_dataset import *
from utils.train_utils import *

In [ ]:
# notebook name
def get_notebook_name():
    import ipynbname
    return ipynbname.name()
nb_fname = get_notebook_name()

In [ ]:
print('PyTorch version:', torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available(): print('cuda is available.')
else: print('cuda is unavailable.') 

-----

## Set the default configuration for building datatset

In [ ]:
cfg_data = {}
cfg_data['device'] = device
cfg_data['dataset'] = 'CAUHS'
cfg_data['data_path'] = r'dataset/02_Curated_Data/'
cfg_data['meta_path'] = os.path.join(cfg_data['data_path'], 'metadata_debug.json')
cfg_data['target_task'] = 'Normal, MCI, Dementia' # 'Norml, MCI, Dementia'
cfg_data['vascular'] = 'X'
cfg_data['segment'] = 'no' # 'train', 'all', 'no'
cfg_data['seed'] = 0
cfg_data['crop_length'] = 200 * 10 # 10 seconds
cfg_data['longer_crop_length'] = 200 * 10 * 10 # 100 seconds
cfg_data['input_norm'] = 'dataset' # 'datatset', 'datapoint', 'no'
cfg_data['EKG'] = 'O'
cfg_data['photic'] = 'X'
cfg_data['awgn'] = 5e-2
cfg_data['awgn_age'] = 5e-2
cfg_data['minibatch'] = 32

In [ ]:
_ = build_dataset(cfg_data, verbose=True)

-----

## Define Network Models

In [ ]:
cfg_common_model = {'in_channels': _[0].dataset[0]['signal'].shape[0], 
                    'out_dims': len(_[-1])}
cfg_model_pool = []

#### 1D Tiny CNN

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Tiny-CNN'
cfg_model['generator'] = TinyCNN1D
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### M7 model (fc-age)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Mx'
cfg_model['generator'] = M7
cfg_model['fc_stages'] = 1
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 1D ResNet model (fc-age)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-2x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### Deeper 1D ResNet model (fc-age)

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### Shallower 1D ResNet

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-2x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### Tiny 1D ResNet model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNet-1x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BasicBlock1D
cfg_model['conv_layers'] = [1, 1, 1, 1]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### Multi-Dilated 1D ResNet model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-Multi-Dilated-ResNet-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = MultiBottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 32
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 1D ResNeXt-53

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-5x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 1D ResNeXt-103

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-ResNeXt-10x'
cfg_model['generator'] = ResNet1D
cfg_model['block'] = BottleneckBlock1D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['groups'] = 32
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')
    
model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 2D ResNet-20 model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-2x' # resnet-18 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = BasicBlock2D
cfg_model['conv_layers'] = [2, 2, 2, 2]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['n_fft'] = 100
cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
cfg_model['hop_length'] = cfg_model['n_fft'] // 2
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 2D ResNet-52 model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNet-5x' # resnet-50 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 6, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['n_fft'] = 100
cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
cfg_model['hop_length'] = cfg_model['n_fft'] // 2
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### 2D ResNeXt-104 model

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '2D-ResNeXt-10x' # resnet-101 + three more fc layer
cfg_model['generator'] = ResNet2D
cfg_model['block'] = Bottleneck2D
cfg_model['conv_layers'] = [3, 4, 23, 3]
cfg_model['fc_stages'] = 3
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 64
cfg_model['n_fft'] = 100
cfg_model['complex_mode'] = 'as_real' # 'power', 'remove'
cfg_model['hop_length'] = cfg_model['n_fft'] // 2
cfg_model['groups'] = 32
cfg_model['width_per_group'] = 8
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### CNN-Transformer

In [ ]:
cfg_model = {}
cfg_model.update(cfg_common_model)
cfg_model['model'] = '1D-CNN-Transformer'
cfg_model['generator'] = CNNTransformer
cfg_model['fc_stages'] = 2
cfg_model['use_age'] = 'fc'
cfg_model['final_pool'] = 'max'
cfg_model['base_channels'] = 256
cfg_model['n_encoders'] = 4
cfg_model['n_heads'] = 4
cfg_model['dropout'] = 0.2
cfg_model['LR'] = 1e-3

pprint.pprint('Model config:')
pprint.pprint(cfg_model)
print('\n' + '-' * 100 + '\n')

model = cfg_model['generator'](**cfg_model).to(device, dtype=torch.float32)
print(model)
print('\n' + '-' * 100 + '\n')

# tensorboard visualization
# visualize_network_tensorboard(model, train_loader, device, nb_fname, '1D-Tiny-CNN-fc-age')

del model
cfg_model_pool.append(cfg_model)

#### Summarize the loaded models

In [ ]:
for cfg_model in cfg_model_pool:
    pprint.pp(cfg_model, width=150)
    print('\n' + '-' * 100 + '\n')

-----

## Default Configurations for Training

In [ ]:
# training configurations
cfg_train = {}
cfg_train['iterations'] = 100000
cfg_train['history_interval'] = cfg_train['iterations'] // 500
cfg_train['lr_decay_step'] = round(cfg_train['iterations'] * 0.8)
cfg_train['lr_decay_gamma'] = 0.1
cfg_train['weight_decay'] = 1e-2
cfg_train['mixup'] = 0.0 # 0 for no usage
cfg_train['criterion'] = 'cross-entropy' # 'cross-entropy', 'multi-bce'

cfg_train['device'] = device
cfg_train['save_model'] = True
cfg_train['save_temporary'] = False
cfg_train['draw_result'] = True
cfg_train['watch_model'] = True

In [ ]:
def train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, class_label_to_type):
    print('*'*120)
    print(f'{"*"*30}{config["model"] + " train starts":^60}{"*"*30}')
    print('*'*120)

    # generate model and its trainer        
    model = config['generator'](**config).to(device)
    
    optimizer = optim.AdamW(model.parameters(), 
                            lr=config['LR'], 
                            weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.StepLR(optimizer, 
                                          step_size=config['lr_decay_step'], 
                                          gamma=config['lr_decay_gamma'])
    
    tr_ms = train_multistep if config.get('mixup', 0) < 1e-12 else train_mixup_multistep
    
    # track granients and weights statistics
    if config.get('watch_model', None):
        wandb.watch(model, log='all', 
                    log_freq=config['history_interval'], 
                    log_graph=True)

    # train and validation routine
    best_val_acc = 0
    for i in range(0, config["iterations"], config["history_interval"]):
        # train 'history_interval' steps
        loss, train_acc = tr_ms(model, train_loader, optimizer, scheduler, 
                                config, config["history_interval"])

        # validation
        val_acc, _, _, _, _ = check_accuracy(model, val_loader, config, repeat=10)

        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_model_state = deepcopy(model.state_dict())
            if config['save_model'] and config['save_temporary']:
                save_path = f'checkpoint_temp/{wandb.run.name}/'
                os.makedirs(save_path, exist_ok=True)
                path = os.path.join(save_path, f'{config["model"]}')
                torch.save(best_model_state, path)

        # log
        wandb.log({'Loss': loss, 
                   'Train Accuracy': train_acc, 
                   'Validation Accuracy': val_acc}, step=i)

    # calculate the test accuracies for best and last models
    last_model_state = deepcopy(model.state_dict())
    last_test_result = check_accuracy(model, test_loader, config, repeat=30)
    last_test_acc = last_test_result[0]

    model.load_state_dict(best_model_state)
    best_test_result = check_accuracy(model, test_loader, config, repeat=30)
    best_test_acc = best_test_result[0]

    if last_test_acc < best_test_acc:
        model_state = best_model_state
        test_result = best_test_result
    else:
        model_state = last_model_state
        test_result = last_test_result

    model.load_state_dict(model_state)
    test_acc, test_confusion, test_debug, score, target = test_result

    # calculate the test accuracies for final model on much longer sequence
    last_test_result = check_accuracy(model, test_loader_longer, config, repeat=30)
    longer_test_acc = last_test_result[0]

    # save the model
    if config['save_model']:
        save_path = f'checkpoint_temp/{wandb.run.name}/'
        os.makedirs(save_path, exist_ok=True)
        path = os.path.join(save_path, f'{config["model"]}')
        torch.save(model_state, path)

    # leave the message
    wandb.config.final_shape = model.get_final_shape()
    wandb.config.num_params = count_parameters(model)
    wandb.log({'Test Accuracy': test_acc,
               '(Best / Last) Test Accuracy': ('Best' if last_test_acc < best_test_acc else 'Last', 
                                               round(best_test_acc, 2), round(last_test_acc, 2)),
               'Confusion Matrix (Array)': test_confusion,
               'Test Accuracy (Longer)': longer_test_acc, 
               'Test Debug Table/Serial': test_debug[0], 
               'Test Debug Table/EDF': test_debug[1], 
               'Test Debug Table/Pred': test_debug[2], 
               'Test Debug Table/GT': test_debug[3]})

    if 'lr_search' in config:
        draw_learning_rate_record(config['lr_search'], use_wandb=True)

    if config['draw_result']:
        draw_roc_curve(score, target, class_label_to_type, use_wandb=True)
        draw_confusion(test_confusion, class_label_to_type, use_wandb=True)
        draw_debug_table(test_debug, use_wandb=True)
        wandb.log({"Confusion Matrix": wandb.plot.confusion_matrix(y_true=target, 
                                                                   preds=score.argmax(axis=-1), 
                                                                   class_names=class_label_to_type)})
        wandb.log({"ROC Curve": wandb.plot.roc_curve(target, score, labels=class_label_to_type)})
        
    return model

In [ ]:
def train_sweep(cfg_data, cfg_train, cfg_model_pool):
    wandb_run = wandb.init()
    wandb.run.name = wandb.run.id
    with wandb_run:
        # wandb config update
        cfg_model = cfg_model_pool[wandb.config.model_index]
        config = {}
        for k, v in {**cfg_data, **cfg_train, **cfg_model}.items():
            if k not in wandb.config:
                config[k] = v
        
        # to prevent callables from type-conversion to str
        wandb.config.update(config)
        for k, v in wandb.config.items():
            if k not in config:
                config[k] = v
                
        # build dataset
        train_loader, val_loader, test_loader, test_loader_longer, class_label_to_type = build_dataset(config)
        
        config['in_channels'] = train_loader.dataset[0]['signal'].shape[0]
        config['out_dims'] = len(class_label_to_type)
        
        # learning rate search if needed
        if config["LR"] is None:
            config['LR'], config['lr_search'] = learning_rate_search(config, train_loader, 
                                                                     min_log_lr=-4.5, max_log_lr=-3.0, 
                                                                     trials=100, steps=100)
																	 
        # train the model
        train_with_wandb(config, train_loader, val_loader, test_loader, test_loader_longer, class_label_to_type)

-----

## Train

In [ ]:
sweep_data = {}
sweep_data['crop_length'] = {
    'values': [200 * 10, # 10 sec
               200 * 20, # 20 sec
              ],
}

sweep_data['EKG'] = {
    'values': ['O', 'X'],
}

sweep_data['photic'] = {
    'values': ['O', 'X'],
}

sweep_data['awgn'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['awgn_age'] = {
    'distribution': 'uniform',
    'min': 0,
    'max': 0.3,
}

sweep_data['minibatch'] = {
    'values': [32, ],
}

In [ ]:
sweep_model = {}
sweep_model['model_index'] = { 
    'values' : [i for i in range(len(cfg_model_pool))] 
}

sweep_model['fc_stages'] = { 
    'distribution' : 'int_uniform',
    'min': 0,
    'max': 4,
}

sweep_model['use_age'] = { 
    'values' : ['fc', 'conv']
}

sweep_model['final_pool'] = { 
    'values' : ['max', 'average']
}

sweep_model['first_dilation'] = { 
    'distribution' : 'int_uniform',
    'min': 1,
    'max': 2,
}

sweep_model['base_stride'] = { 
    'distribution' : 'int_uniform',
    'min': 2,
    'max': 4,
}

sweep_model['dropout'] = {
    'distribution': 'uniform',
    'min': 0.0,
    'max': 0.5
}

sweep_model['LR'] = {
    'distribution': 'log_uniform',
    'min': math.log(1e-5),
    'max': math.log(1e-3)
}

In [ ]:
sweep_train = {}
sweep_train['iterations'] = {
    'values' : [100000, 150000]
}

sweep_train['lr_decay_gamma'] = {
    'distribution' : 'uniform',
    'min': 0.1,
    'max': 0.5,
}

sweep_train['lr_decay_step'] = {
    'values' : [45000, 80000]
}

sweep_train['weight_decay'] = {
    'distribution' : 'log_uniform',
    'min': math.log(1e-5),
    'max': math.log(1e-1)
}

sweep_train['mixup'] = {
    'values': [0, 0.15, 0.3]
}

sweep_train['criterion'] = {
    'values': ['cross-entropy', 'multi-bce']
}

In [ ]:
sweep_config = {
    "entity": "ipis-mjkim",
    "name" : "my-sweep",
    "method" : "random",
    "parameters" : 
    {
        **sweep_data,
        **sweep_model,
        **sweep_train,
    }
}

sweep_id = wandb.sweep(sweep_config, project="eeg-analysis")

In [ ]:
wandb.agent(sweep_id, function=lambda: train_sweep(cfg_data, cfg_train, cfg_model_pool), count=7)